
# 1. Настройка: установка пакета.


In [1]:
# Устанавливаем необходимые библиотеки. Используем --upgrade, чтобы убедиться, что это последние версии.
# Добавляем fsspec, чтобы явно обновить его до последней версии
!pip install --upgrade --quiet datasets evaluate transformers sentencepiece accelerate fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Задание 2: Загрузка и изучение набора данных.

In [2]:
from datasets import load_dataset
import pandas as pd # Обычно pandas импортируют как pd, для краткости

# Загружаем UCI SMS Spam датасет (sms_spam) с Hugging Face Hub
raw_dataset = load_dataset("sms_spam", revision='main') # Используем raw_dataset, чтобы не конфликтовать с 'raw' из предыдущих ошибок

# Мы будем использовать 4,000 для обучения, 1,000 для валидации
# Обрати внимание, что датасет "sms_spam" содержит только одну часть "train".
# Мы вручную разделяем ее на train и validation.
train_ds = raw_dataset["train"].select(range(4000))
val_ds   = raw_dataset["train"].select(range(4000, 5000))

# Выводим признаки (features) тренировочного набора данных. Должно быть 'sms' и 'label'.
print("--- Признаки тренировочного набора данных ---")
print(train_ds.features)

# Выведем первые несколько примеров, чтобы понять структуру данных
print("\n--- Первые 5 примеров тренировочного набора данных ---")
print(train_ds.to_pandas().head())

# Посмотрим на распределение классов
print("\n--- Распределение классов в тренировочном наборе данных ---")
print(pd.Series(train_ds['label']).value_counts())
print("\n--- Распределение классов в валидационном наборе данных ---")
print(pd.Series(val_ds['label']).value_counts())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

--- Признаки тренировочного набора данных ---
{'sms': Value('string'), 'label': ClassLabel(names=['ham', 'spam'])}

--- Первые 5 примеров тренировочного набора данных ---
                                                 sms  label
0  Go until jurong point, crazy.. Available only ...      0
1                    Ok lar... Joking wif u oni...\n      0
2  Free entry in 2 a wkly comp to win FA Cup fina...      1
3  U dun say so early hor... U c already then say...      0
4  Nah I don't think he goes to usf, he lives aro...      0

--- Распределение классов в тренировочном наборе данных ---
0    3466
1     534
Name: count, dtype: int64

--- Распределение классов в валидационном наборе данных ---
0    861
1    139
Name: count, dtype: int64


SUM

датасет несбалансирован по классам, что является типичной ситуацией для обнаружения спама. Это означает, что при оценке модели нам потребуется не только accuracy (точность), но и другие метрики, такие как precision, recall и f1-score, чтобы получить полное представление о производительности модели.

The dataset is unbalanced across classes, which is a typical situation for spam detection. This means that when evaluating a model, we will need not only accuracy, but also other metrics such as precision, recall, and f1-score to get a full picture of the model's performance.

# 3. Tokenization

In [3]:
from transformers import GPT2TokenizerFast, AutoTokenizer # GPT2TokenizerFast - современный токенизатор для GPT2

model_name = "gpt2" # Используем GPT-2, как указано в задании
tokenizer  = AutoTokenizer.from_pretrained(model_name) # Загружаем токенизатор

# GPT-2 по умолчанию не имеет pad_token. Для классификации последовательностей нам нужен pad_token.
# Устанавливаем его равным eos_token (конец последовательности), это обычная практика для GPT-подобных моделей в задачах классификации.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(examples):
    # Возвращает input_ids, attention_mask; max_length для SMS-сообщений можно сделать небольшим
    # GPT-2 не использует token_type_ids для классификации, поэтому их не включаем
    return tokenizer(
        examples["sms"],
        padding="max_length", # Дополняем до максимальной длины батча или до max_length
        truncation=True,      # Обрезаем, если текст длиннее max_length
        max_length=64         # Максимальная длина последовательности (в токенах)
    )

# Применяем токенизацию к тренировочному и валидационному наборам данных
# .map() применяет функцию к каждой записи датасета и возвращает новый датасет
train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok   = val_ds.map(tokenize_fn, batched=True)

# Чтобы модель GPT2ForSequenceClassification работала корректно,
# нам нужны столбцы 'input_ids', 'attention_mask' и 'labels'.
# Мы удалим оригинальный столбец 'sms', так как он больше не нужен после токенизации,
# и переименуем 'label' в 'labels' (это стандартное имя, ожидаемое Trainer-ом).
train_tok = train_tok.remove_columns(["sms"]).rename_column("label", "labels")
val_tok   = val_tok.remove_columns(["sms"]).rename_column("label", "labels")

# Также установим формат PyTorch тензоров, так как модель будет работать с PyTorch
train_tok.set_format("torch")
val_tok.set_format("torch")

print("\n--- Пример токенизированного тренировочного примера ---")
print(train_tok[0])
print(f"Длина input_ids первого примера: {len(train_tok[0]['input_ids'])}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


--- Пример токенизированного тренировочного примера ---
{'labels': tensor(0), 'input_ids': tensor([ 5247,  1566,  8174,   506,   966,    11,  7165,   492, 14898,   691,
          287,  5434,   271,   299,  1049,   995,  8591,   304, 44703,   986,
          327,   500,   612,  1392,   716,   382,  4383,   986,   198, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
Длина input_ids первого примера: 64


# Task 4: Initialization of the model.

In [5]:
import torch
from transformers import GPT2ForSequenceClassification # Импортируем модель для классификации

model = GPT2ForSequenceClassification.from_pretrained( # Загружаем GPT-2 с классификационной головой
    model_name,                            # Используем 'gpt2', как определено ранее
    num_labels=2,                          # 2 метки: спам (1) vs. не-спам (0)
    pad_token_id=tokenizer.eos_token_id,   # Устанавливаем pad_token_id, как мы делали для токенизатора
    # Также полезно указать id2label и label2id для лучшей читаемости результатов,
    # хотя это не строго обязательно для работы модели
    id2label={0: "ham", 1: "spam"},
    label2id={"ham": 0, "spam": 1}
)

# Перемещаем модель на GPU, если доступно, для ускорения обучения
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"\nМодель '{model_name}' для классификации загружена и готова к обучению на {device}!")
print("Структура классификационного слоя модели:")
print(model.score) # ИСПРАВЛЕНИЕ: ИСПОЛЬЗУЕМ .score вместо .classifier

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Модель 'gpt2' для классификации загружена и готова к обучению на cuda!
Структура классификационного слоя модели:
Linear(in_features=768, out_features=2, bias=False)


# 5. Metrics Definition

In [6]:
import evaluate
import numpy as np

# Загружаем объекты метрик из библиотеки 'evaluate'
accuracy  = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall    = evaluate.load("recall")
f1        = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)

    return {
        "accuracy":  accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall":    recall.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1":        f1.compute(predictions=preds, references=labels, average="binary")["f1"]
    }

# 6. Configuration

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",        # Директория для сохранения контрольных точек и логов
    do_train=True,                 # Включаем тренировку
    do_eval=True,                  # Включаем оценку
    eval_steps=500,                # Запускать .evaluate() каждые 500 шагов
    save_steps=500,                # Сохранять чекпоинт каждые 500 шагов
    logging_dir="./logs",          # Директория для логов TensorBoard
    logging_steps=500,             # Логировать метрики каждые 500 шагов

    per_device_train_batch_size=8, # Размер батча на GPU/CPU для обучения
    per_device_eval_batch_size=8,  # Размер батча на GPU/CPU для оценки
    num_train_epochs=3.0,          # Количество полных проходов по тренировочному датасету
    learning_rate=5e-5,            # Скорость обучения (часто 5e-5 для Fine-tuning)
    weight_decay=0.01,             # Параметр для регуляризации L2 (предотвращает переобучение)

    report_to="none",              # Отключаем интеграции с W&B, TensorBoard и т.д.
    save_total_limit=1,            # Оставлять только последний чекпоинт
    # load_best_model_at_end=True, # Эти аргументы появились в более поздних версиях, поэтому убираем их
    # metric_for_best_model="f1",
    # greater_is_better=True,
)

# 7. Training and assessment

In [9]:
from transformers import Trainer

# Инициализируем Trainer
trainer = Trainer(
    model=model,                 # Наша модель GPT2ForSequenceClassification
    args=training_args,          # Настроенные аргументы обучения
    train_dataset=train_tok,     # Токенизированный тренировочный набор данных
    eval_dataset=val_tok,        # Токенизированный валидационный набор данных
    compute_metrics=compute_metrics, # Функция для вычисления метрик
)

# Запускаем процесс обучения
print("\n--- Запуск обучения модели ---")
trainer.train()

# Оцениваем модель после обучения на валидационном наборе данных
print("\n--- Оценка модели на валидационном наборе данных ---")
metrics = trainer.evaluate()
print(metrics)

# Ожидаемый результат: {"eval_loss": ..., "eval_accuracy": 0.98, ...}
# Обрати внимание, что метрики, которые мы определили в compute_metrics,
# также будут отображены (eval_precision, eval_recall, eval_f1).


--- Запуск обучения модели ---


Step,Training Loss
500,0.164200
1000,0.052800
1500,0.022200



--- Оценка модели на валидационном наборе данных ---


{'eval_loss': 0.04992344602942467, 'eval_accuracy': 0.993, 'eval_precision': 0.9925373134328358, 'eval_recall': 0.9568345323741008, 'eval_f1': 0.9743589743589743, 'eval_runtime': 4.0105, 'eval_samples_per_second': 249.348, 'eval_steps_per_second': 31.168, 'epoch': 3.0}


SUM
Training Loss:

500: 0.164200

1000: 0.052800

1500: 0.022200

Interpretation: The loss value keeps decreasing throughout training. This is a great sign that the model is successfully learning from the training data and getting better at predicting labels.

eval_loss:

0.0499

Interpretation: The low loss on the validation dataset indicates that the model generalizes well to new, unseen messages, rather than simply "memorizing" the training ones.

eval_accuracy:

0.993 (or 99.3%)

Interpretation: The model correctly classifies 99.3% of all messages in the validation dataset. This is a very high overall accuracy!

eval_precision (Precision for the "spam" class):

0.9925 (or 99.25%)

Interpretation: Of all the messages that the model marked as spam, 99.25% were actually spam. This is a fantastic result! This means that very few legitimate messages will be mistakenly marked as spam (a very low false positive rate).

eval_recall (Recall for the "spam" class):

0.9568 (or 95.68%)

Interpretation: The model was able to detect 95.68% of all real spam messages in the validation set. This is also a very good score, meaning that only a small percentage of spam will slip through the cracks.

eval_f1 (F1-score for the "spam" class):

0.9743

Interpretation: The F1-score is the harmonic mean of precision and recall. A high F1 value (close to 1) indicates that the model achieves an excellent balance between precision and recall. This is especially important for imbalanced data, where high accuracy alone can be misleading.

The retrained GPT-2 model demonstrates outstanding performance in classifying SMS spam! Not only is it very accurate overall, but it also shows strong precision and recall for the spam class, which is critical given the imbalanced nature of the data. This means that the model:

Will be virtually free of mislabeling important messages as spam.

Will effectively detect the vast majority of real-world spam messages.